In [ ]:
import time
import sys
import os
import numpy as np

%load_ext autoreload
%autoreload 2

import xfmkit.utils as utils
import xfmkit.argops as argops
import xfmkit.clustering as clustering
import xfmkit.visualisations as vis
import xfmkit.processops as processops
import xfmkit.structures as structures
import xfmkit.entry_processed as entry_processed

#add the parent folder to sys path so we can import from the notebook subfolder
sys.path.insert(0,'..')

#args = ' -d /home/lachlan/CODEBASE/xfmkit/data/processed_maps/carlos_full -n 2 -fc'
args_in = ["-d", "/home/lachlan/CODEBASE/xfmkit/data/processed_maps/carlos_full", "-n", "2", "-fc" ]

PACKAGE_CONFIG='xfmkit/config.yaml'

In [ ]:
"""
read exported tiffs from geopixe

perform clustering and visualisation
"""
#create input config from args and config files
config =utils.initcfg(PACKAGE_CONFIG)

#get command line arguments
args = argops.readargs_processed(args_in, config)

image_directory=args.input_directory
output_directory=os.path.join(image_directory, "outputs")

if not os.path.isdir(output_directory):
    os.mkdir(output_directory)

ds = processops.compile(image_directory)

ds.crop((args.x_coords[0], args.x_coords[1]), (args.y_coords[0], args.y_coords[1]))

pxs = structures.PixelSet(ds)

In [ ]:
import pandas as pd

IGNORE_LINES=[]
AFFECTED_LINES=['Ar', 'Mo', 'MoL']
LIGHT_LINES=['Mg', 'Al', 'Si', 'P', 'S']
NON_ELEMENT_LINES=['sum','Back','Compton']

AMP_FACTOR=10.0
SUPPRESS_FACTOR=10.0

pixelseries=pxs

q99_set = np.zeros(len(pixelseries.data.d[1]))

amplify=args.amplify
suppress=args.suppress
normalise=args.normalise
data_transform=args.data_transform
weight_transform=args.data_transform


for i, label in enumerate(pixelseries.labels):
    DATA_QUANTILE=0.999
    q99_set[i] = utils.mean_within_quantile(pixelseries.data.d[:,i], qmin=DATA_QUANTILE)


In [ ]:
df = pd.DataFrame(columns=pixelseries.labels)

df.loc[0]=q99_set

df

In [ ]:
df['Mg']

In [ ]:
dfd = df.drop(labels=LIGHT_LINES+NON_ELEMENT_LINES+AFFECTED_LINES, axis=1) 

sorted = dfd.iloc[0].sort_values(ascending=False)

N_TO_AVG=4

sorted[0:N_TO_AVG].mean()


In [ ]:

df = pd.DataFrame(q99_set, columns=pixelseries.labels)

for target in amplify:
    
    for i, label in enumerate(pixelseries.labels):
        if label == target:
            pixelseries.weights[i]=pixelseries.weights[i]*AMP_FACTOR